# Providers and products

Only providers that do not need authentication for search will be listed here. This notebook is executed in CI/CD without providers credentials, and EODAG hides providers needing authentication for search that do not have credentials set.

In [1]:
from eodag import EODataAccessGateway, setup_logging
from eodag.api.product_type import ProductType

setup_logging(2)

dag = EODataAccessGateway()

/home/tlarrouy/miniconda3/envs/ox/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-01 10:59:03,550 eodag.config                     [INFO    ] Loading user configuration from: /home/tlarrouy/.config/eodag/eodag.yml
2025-08-01 10:59:03,561 eodag.core                       [INFO    ] usgs: provider needing auth for search has been pruned because no credentials could be found
2025-08-01 10:59:03,562 eodag.core                       [INFO    ] aws_eos: provider needing auth for search has been pruned because no credentials could be found
2025-08-01 10:59:03,562 eodag.core                       [INFO    ] cop_ads: provider needing auth for search has been pruned because no credentials could be found
2025-08-01 10:59:03,562 eodag.core                       [INFO    ] cop_cds: provider needing auth for 

## Providers available

The method [available_providers()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.available_providers) returns a list of the pre-configured providers.

In [2]:
available_providers = dag.available_providers()
available_providers

['peps',
 'cop_dataspace',
 'cop_marine',
 'creodias',
 'dedl',
 'earth_search',
 'earth_search_cog',
 'earth_search_gcs',
 'ecmwf',
 'eumetsat_ds',
 'geodes',
 'planetary_computer',
 'sara',
 'theia',
 'usgs_satapi_aws']

In [3]:
print(f"eodag has {len(available_providers)} providers already configured.")

eodag has 15 providers already configured.


It can take a product type as an argument and will return the providers known to `eodag` that offer this product.

In [4]:
dag.available_providers("S2_MSI_L1C")

['peps',
 'cop_dataspace',
 'creodias',
 'dedl',
 'earth_search',
 'earth_search_gcs',
 'geodes',
 'sara']

<div class="alert alert-warning">

Note

If a provider is configured to need authentication for search, and has no credentials set, it will be pruned on EODAG initialization, and will not appear in available providers list.

</div>

## Product types available

The method [list_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types) returns a [ProductTypesList](../../api_reference/product_type.rst#eodag.api.product_type.ProductTypesList) instance that represents `eodag`'s internal product type catalog if used with `fetch_providers=False`. It will fetch providers for new product types and return an extended list if used with `fetch_providers=True` (default behavior).

In [5]:
internal_catalog = dag.list_product_types(fetch_providers=False)
print(f"EODAG has {len(internal_catalog)} product types stored in its internal catalog.")

EODAG has 275 product types stored in its internal catalog.


In [6]:
extended_catalog = dag.list_product_types()
print(f"EODAG has {len(extended_catalog)} product types stored in its extended catalog, after having fetched providers.")

2025-08-01 10:59:03,867 eodag.config                     [INFO    ] Fetching external product types from https://cs-si.github.io/eodag/eodag/resources/ext_product_types.json
2025-08-01 10:59:08,304 eodag.search.qssearch            [INFO    ] Fetching product types: https://hda.data.destination-earth.eu/stac/collections


EODAG has 911 product types stored in its extended catalog, after having fetched providers.


When providers are fetched for new product types, `eodag`'s product types configuration is updated in `EODataAccessGateway` instance. Extended product types list is then returned independantly of `fetch_providers` option in [list_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types):

In [7]:
called_again_catalog = dag.list_product_types(fetch_providers=False)
print(f"list_product_types() keeps returning {len(called_again_catalog)} product types.")

list_product_types() keeps returning 911 product types.


In [8]:
internal_catalog[0]

ProductType(id='AERIS_IAGOS', title='In-service Aircraft for a Global Observing System', missionStartDate='1994-08-01T00:00:00Z', processingLevel='L2', instrument='IAGOS-CORE,IAGOS-MOZAIC,IAGOS-CARIBIC', sensorType='ATMOSPHERIC', keywords='AERIS, AIRCRAFT, ATMOSPHERIC, IAGOS, L2', license='other', abstract='The mission of IAGOS is to provide high quality data throughout the tropopshere\nand lower stratosphere, and scientific expertise to understand the evolution of\natmospheric composition, air quality, and climate.\n')

In [11]:
products_id = [p.id for p in internal_catalog]
products_id

['AERIS_IAGOS',
 'CAMS_EAC4',
 'CAMS_EAC4_MONTHLY',
 'CAMS_EU_AIR_QUALITY_FORECAST',
 'CAMS_EU_AIR_QUALITY_RE',
 'CAMS_GAC_FORECAST',
 'CAMS_GFE_GFAS',
 'CAMS_GLOBAL_EMISSIONS',
 'CAMS_GREENHOUSE_EGG4',
 'CAMS_GREENHOUSE_EGG4_MONTHLY',
 'CAMS_GREENHOUSE_INVERSION',
 'CAMS_GRF',
 'CAMS_GRF_AUX',
 'CAMS_SOLAR_RADIATION',
 'CLMS_CORINE',
 'CLMS_GLO_DMP_333M',
 'CLMS_GLO_FAPAR_333M',
 'CLMS_GLO_FCOVER_333M',
 'CLMS_GLO_GDMP_333M',
 'CLMS_GLO_LAI_333M',
 'CLMS_GLO_NDVI_1KM_LTS',
 'CLMS_GLO_NDVI_333M',
 'COP_DEM_GLO30_DGED',
 'COP_DEM_GLO30_DTED',
 'COP_DEM_GLO90_DGED',
 'COP_DEM_GLO90_DTED',
 'DT_CLIMATE_ADAPTATION',
 'DT_EXTREMES',
 'EEA_DAILY_VI',
 'EFAS_FORECAST',
 'EFAS_HISTORICAL',
 'EFAS_REFORECAST',
 'EFAS_SEASONAL',
 'EFAS_SEASONAL_REFORECAST',
 'ERA5_LAND',
 'ERA5_LAND_MONTHLY',
 'ERA5_PL',
 'ERA5_PL_MONTHLY',
 'ERA5_SL',
 'ERA5_SL_MONTHLY',
 'EUSTAT_AVAILABLE_BEDS_HOSPITALS_NUTS2',
 'EUSTAT_BATHING_SITES_WATER_QUALITY',
 'EUSTAT_GREENHOUSE_GAS_EMISSION_AGRICULTURE',
 'EUSTAT_POP_A

The method can take a provider name as an argument and will return the product types known to `eodag` that are offered by this provider.

In [9]:
peps_products = dag.list_product_types("peps")
[p.id for p in peps_products]

['S1_SAR_GRD', 'S1_SAR_OCN', 'S1_SAR_SLC', 'S2_MSI_L1C']

## Combine these two methods

These two methods can be combined to find which product type is the most common in `eodag`'s catalog among all the providers.

In [12]:
availability_per_product = []
for product in products_id:
    providers = dag.available_providers(product)
    availability_per_product.append((product, len(providers)))
availability_per_product = sorted(availability_per_product, key=lambda x: x[1], reverse=True)
most_common_p_type, nb_providers = availability_per_product[0]
print(f"The most common product type is '{most_common_p_type}' with {nb_providers} providers offering it.")

The most common product type is 'S2_MSI_L1C' with 11 providers offering it.


These can be also used to find out which provider (as configured by `eodag`) offers the hights number of different product types.

In [13]:
availability_per_provider = []
for provider in dag.available_providers():
    provider_products_id = [
        p.id
        for p in dag.list_product_types(provider, fetch_providers=False)
    ]
    availability_per_provider.append(
        (provider, len(provider_products_id))
    )
availability_per_provider = sorted(availability_per_provider, key=lambda x: x[1], reverse=True)
provider, nb_p_types = availability_per_provider[0]
print(f"The provider with the largest number of product types is '{provider}' with {nb_p_types}.")

The provider with the largest number of product types is 'wekeo_cmems' with 796.


## Search on a product type

The common way to search products is to use the [search()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.search) gateway method. However, a new [search()](../../api_reference/product_type.rst#eodag.api.product_type.ProductType.search) method have been implemented from [ProductType](../../api_reference/product_type.rst#eodag.api.product_type.ProductType) instances to directly search on the product type wanted by the user. It works the same way except the gateway must be given as parameter to keep the `eodag` and the user configurations.

In [14]:
products_first_page = ProductType(id="S2_MSI_L1C").search(dag, start="2021-03-01", end="2021-03-31")

2025-08-19 11:04:24,092 eodag.core                       [INFO    ] Searching on provider peps
2025-08-19 11:04:24,092 eodag.search.qssearch            [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2021-03-01&completionDate=2021-03-31&productType=S2MSI1C&maxRecords=20&page=1


In [15]:
print(f"Got a hand on {len(products_first_page)} products.")

Got a hand on 20 products.


## List queryables on a product type

Like for the [search()](../../api_reference/product_type.rst#eodag.api.product_type.ProductType.search) method, a new [list_queryables()](../../api_reference/product_type.rst#eodag.api.product_type.ProductType.list_queryables) method have been implemented from [ProductType](../../api_reference/product_type.rst#eodag.api.product_type.ProductType) instances to directly list queryables on the product type wanted by the user.

In [16]:
ProductType(id="S2_MSI_L1C").list_queryables(dag)

2025-08-19 11:04:31,635 eodag.search.qssearch            [INFO    ] Fetching queryables: https://gateway.prod.wekeo2.eu/hda-broker/api/v1/dataaccess/queryable/EO:ESA:DAT:SENTINEL-2
2025-08-19 11:04:32,548 eodag.search.qssearch            [INFO    ] Fetching queryables: https://hda.data.destination-earth.eu/stac/search/../collections/EO.ESA.DAT.SENTINEL-2.MSI.L1C/queryables


QueryablesDict (42) - additional_properties=True
"str,"
"FieldInfo(annotation=NoneType, required=False, default=None, alias='startTimeFromAscendingNode', alias_priority=2, description=""Date/time as string in ISO 8601 format (e.g. '2024-06-10T12:00:00Z')"")"
"str,"
"FieldInfo(annotation=NoneType, required=False, default=None, alias='completionTimeFromAscendingNode', alias_priority=2, description=""Date/time as string in ISO 8601 format (e.g. '2024-06-10T12:00:00Z')"")"
"str,"
"FieldInfo(annotation=NoneType, required=False, default=None)"
"str,"
"FieldInfo(annotation=NoneType, required=False, default=None)"
"str,"
"FieldInfo(annotation=NoneType, required=False, default=None)"


## Product types discovery

EODAG comes with a large list of pre-configured product types. Some others are available from providers catalogs but will not be configured, or are not yet configured in EODAG.

Some providers, like STAC providers, come in EODAG with a configuration describing how to discover these not-already-configured product types.

With the method [discover_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.discover_product_types) 
or CLI command [eodag discover](../../cli_user_guide.rst) we can obtain a JSON configuration file that will be used as *EODAG 
external product types configuration file*.

In EODAG, the discovered *EODAG external product types configuration file* can be set to:

* a file automatically built from github actions and stored in [eodag/resources/ext_product_types.json](https://cs-si.github.io/eodag/eodag/resources/ext_product_types.json) (default settings)
* a custom remote or local file by setting its path in `EODAG_EXT_PRODUCT_TYPES_CFG_FILE` environment variable (if the file is not readable, only user-modified providers will be fetched).

Then, when listing product types using [list_product_types(fetch_providers=True)](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types), EODAG will first read the content of the *EODAG external product types configuration file* using [fetch_product_types_list()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.fetch_product_types_list) 
then update [EODataAccessGateway](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway) instance product types configuration, if needed.

The obtained product types list will contain both pre-configured and discovered product types.

![Fetch product types schema](../../_static/eodag_fetch_product_types.png "Fetch product types schema")